In [2]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from io import BytesIO  
%matplotlib inline
#To compute intersection of lines
import shapely
from shapely.geometry import LineString, Point
from scipy.spatial import distance

In [3]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];

CONNECTED_EYELINE_NAMES = [
    ("rightShoulder", 'leftEye') , ('rightEar', 'rightEye')
]
CONNECTED_EYELINE_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_EYELINE_NAMES];

In [20]:
len(PART_NAMES)

17

#Helper functions

In [4]:
def getBase64Image(filePath):
    with open(filePath, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        my_string = my_string.decode('utf-8')
    return my_string

In [5]:
def convertToBase64(image):
    retval, buffer = cv2.imencode('.jpg', image)
    my_string = base64.b64encode(buffer)
    my_string = my_string.decode('utf-8')
    return my_string

In [6]:
def get_adjacent_keypoints(keypoints):
    '''
    Helper function of draw_skel_and _kp
    Returns 2 coord of 2 points where line needs to be drawn
    EXAMPLE: [[X1,Y1],[X2,Y2]]
    '''
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            

In [7]:
def get_adjacent_keypoint_eyeline(keypoints):
    results = []
    for left, right in CONNECTED_EYELINE_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results

In [8]:
#Sending data to server
def getPoses(image_string):
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+image_string}
    r = req.post(url=url2 , data = data)
    poses = r.json()
    return poses

In [9]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 

In [10]:
def draw_skelton(poses,img):
    '''
    This function is not used anywhere. 
    Written to demonstrate how skelton can be drawn
    '''
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [11]:
def draw_skel_and_kp(poses , img, color):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    #For every pose of the player
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=color,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=list(color))
    return out_img

In [12]:
def draw_extended_line(out_img,p1,p2,color):
    '''
    Helper function of draw_visibility_cone
    Given 2 points and 1 image, draw a lines extends it
    '''
    theta1 = np.arctan2(p1[1]-p2[1], p1[0]-p2[0]) 
    endpt_x = int(p1[0] - 1000*np.cos(theta1))
    endpt_y = int(p1[1] - 1000*np.sin(theta1))
    cv2.line(out_img, (p1[0], p1[1]), (endpt_x, endpt_y), color, 2)
    
    return out_img,theta1,[endpt_x, endpt_y]
    

In [13]:
def get_line_intersection(p1,p2,p3,p4):
    '''
    Helper function of drawvisibilty_cone
    '''
    try:
        line1 = LineString([[p1[0] , p1[1]], [p2[0] , p2[1]] ])
        line2 = LineString([[p3[0] , p3[1]], [p4[0] , p4[1]] ])    
        int_pt = line1.intersection(line2)
        #print(int_pt)
        p5 = [int_pt.x, int_pt.y]        
    except AttributeError:
        #print("Returing None")
        return None
    return p5

In [14]:
def find_angle_between_2_line(p1,p2,p3,p4):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p4)
    
    cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle = np.arccos(cosine_angle)
    return angle

In [15]:
def draw_general_line(out_img,p5,theta3,color):
    '''
    Helper functino of draw_visibilty_cone
    
    '''
    endpt_x = int(p5[0] - 1000*np.cos(theta3))
    endpt_y = int(p5[1] - 1000*np.sin(theta3))
    cv2.line(out_img, (int(p5[0]), int(p5[1])), (endpt_x, endpt_y), color, 2)
    return out_img

In [16]:
def get_positive_theta(theta):
    if 0<=theta<=np.pi:
        return theta
    else:
        if -np.pi/2 <=theta < 0:
            theta = 2*np.pi - abs(theta)
        elif -np.pi <= theta < np.pi/2:
            theta = (3/4*np.pi) - abs(theta)
    return theta
            
        

In [17]:
def draw_visibilty_cone(poses,img,color=None):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    counter = 0
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoints = get_adjacent_keypoint_eyeline(keypoints)
        p1,p2 = new_keypoints[0][0],new_keypoints[0][1]
        p3,p4 = new_keypoints[1][0],new_keypoints[1][1]
       # Draw the two lines
        out_img,theta1,p2 = draw_extended_line(out_img,p1,p2,[255,255,0])
        out_img,theta2,p4= draw_extended_line(out_img,p3,p4,[0,255,0])
       # get mid line 
#         p5 = get_line_intersection(p1,p2,p3,p4)
#         if p5 != None:
#             theta3 = (theta1 + theta2)/2
#             if counter <100:
#                 draw_general_line(out_img,p5,theta3,[0,0,255])
#             counter += 1
    return out_img

In [17]:
p5 = [536.3247184387701, 328.8307943698096]
draw_general_line(out_img,p5,np.radians(120.06858282186246),[255,0,255])
cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/cavsCone.JPG' ,out_img) 

NameError: name 'out_img' is not defined

#Sending to server

In [18]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')

In [18]:

img = cv2.imread('/Users/sandeep/Desktop/dataandmodles/data/cropped.png')
myString =convertToBase64(img)
poses = getPoses(myString)
out_img = draw_skel_and_kp(poses,img,[255,0,0])
#out_img = draw_visibilty_cone(poses,img)
cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/croppedPose.JPG' ,out_img) 

True

In [24]:
videoPath ='/Users/sandeep/Desktop/dataandmodles/data/shoot.mov'
cap = cv2.VideoCapture(videoPath)
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    if ret:
        #do stuff
        myString =convertToBase64(frame)
        poses = getPoses(myString)
        #frame = draw_visibilty_cone(poses,frame)
        frame = draw_skel_and_kp(poses,frame,[255,0,0])
#         End of do stuff
        cv2.imshow('test', frame)
        #cv2.imwrite(f"/Users/sandeep/Desktop/dataandmodles/data/coneVision/{count}.png" , frame)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  
    else:
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()
